In [8]:
import numpy as np
from state_of_the_art.tables.insights_table import InsightsTable
from state_of_the_art.tables.text_feedback_table import TextFeedbackTable
InsightsTable

df = TextFeedbackTable().read().sort_values(by='tdw_timestamp', ascending=False)
df = df[df['score'].notnull()]

df =  df[['text', 'score', 'tdw_uuid']]

df

,text,score,tdw_uuid
0,This methodology can optimize pricing strategi...,2,55ab06ec-1213-44bf-85c5-17f3c8dd1d2f
0,A Markov Chain model can effectively estimate ...,2,039d70cf-953f-4f6d-a5cf-b360a1bd1624
0,Incremental LTV is a more accurate measure as ...,2,9c4e4f33-284f-4040-bc29-d4a857717b42
0,This methodology can optimize pricing strategi...,2,a47e6c55-2459-4fdf-bf74-45f7cb4f4d84
0,A Markov Chain model can effectively estimate ...,2,7f836874-01b5-4032-a609-ba37c4a8caff
...,...,...,...
0,Bridging 4D QFTs and 2D VOAs via 3D high-tempe...,0,62a13b6b-2f4d-4e71-8582-0c5fccc395b9
0,Open-World Evaluation for Retrieving Diverse P...,0,cc94b3f3-f8ed-42cf-a852-26d4f9c091e5
0,Bridging 4D QFTs and 2D VOAs via 3D high-tempe...,0,dc624675-d9ea-4afa-bf60-3438abc7a6b5
0,Bridging 4D QFTs and 2D VOAs via 3D high-tempe...,0,0bad9198-8f85-4f8f-81d3-07a3848641c6


In [11]:

rows_number=len(df) 
test_size = int(rows_number*0.3)
train_size = rows_number - test_size

train = df.sample(train_size, random_state=42)
df_all = df.merge(train.drop_duplicates(), on=['tdw_uuid','tdw_uuid'],
                   how='left', indicator=True, suffixes=('', '_y'))

#create DataFrame with rows that exist in first DataFrame only
test = df_all[df_all['_merge'] == 'left_only']
test = test[[c for c in test.columns if not c.endswith('_y')]]
# drop merge column
test = test.drop(columns=['_merge'])
test

,text,score,tdw_uuid
1,A Markov Chain model can effectively estimate ...,2,039d70cf-953f-4f6d-a5cf-b360a1bd1624
2,Incremental LTV is a more accurate measure as ...,2,9c4e4f33-284f-4040-bc29-d4a857717b42
14,The paper provides valuable recommendations on...,2,0568bebc-97ea-4af8-8308-4cb32282cbc7
20,On Testing Machine Learning Programs,2,e5287254-1433-45ec-b8dc-fc641056487f
21,On Testing Machine Learning Programs,2,0a48f8f7-d26a-4bc0-8357-9018881cebc2
23,On Testing Machine Learning Programs,2,6cebb6ff-d198-448f-bf87-7ddd8cb5d3b0
29,On Testing Machine Learning Programs,2,85ada0a0-425f-4fa5-9981-19bc2e3c0d6a
32,Customized frameworks and continuous risk asse...,1,158a1be9-dbd8-4f19-a4be-e2460fc8ff77
37,Empowering Open Data Sharing for Social Good: ...,1,d6b20068-df67-414e-9df3-c0ca34956d6e
41,Customized frameworks and continuous risk asse...,1,24a27a25-2511-4140-a417-9035fddd620c


In [12]:
import numpy as np


train_y = train['score'].astype('int64').to_numpy()
test_y =  test['score'].astype('int64').to_numpy()
train_y

array([0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 2, 2, 0, 1, 1, 0,
       1, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 1, 2, 2, 2, 1, 2,
       0, 0, 0, 1, 1, 2, 0, 1, 2, 0, 2, 0, 2, 2, 1, 1, 3, 1, 2, 2, 0, 1,
       1, 0, 1, 2, 2, 0, 1, 1, 1, 0, 2, 1, 1, 0, 0, 0, 2, 2, 0, 2, 2, 1,
       1, 0, 2, 0, 0, 0, 1, 2, 1, 2])

In [14]:

from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2")

# The sentences to encode
# 2. Calculate embeddings by calling model.encode()
train_embedding = embeddings = model.encode(train['text'].to_list())
test_embedding = embeddings = model.encode(test['text'].to_list())
train_embedding[0]

array([ 7.89197441e-03,  9.02462751e-02,  2.08357517e-02,  6.52584527e-03,
       -2.03398429e-02, -3.08239814e-02,  4.06673923e-02, -2.50476953e-02,
       -5.76594006e-03, -2.57303752e-02, -4.09281515e-02, -2.42832340e-02,
        1.88407619e-02,  8.04651678e-02, -1.23653654e-02, -6.56040013e-02,
        8.64448398e-03, -2.40873136e-02, -2.49913614e-02, -1.75694544e-02,
       -3.39829698e-02, -3.47140059e-02, -1.96876135e-02,  5.86898103e-02,
        5.88592030e-02, -3.31388079e-02, -1.91522725e-02,  8.62770062e-03,
        7.31828660e-02,  1.93499234e-02, -3.04462221e-02,  2.05373485e-02,
       -3.20900120e-02,  1.03001110e-02,  1.55590089e-06, -4.20467779e-02,
       -5.76686636e-02, -4.44340566e-03,  4.48590815e-02, -2.70418078e-02,
        5.15876226e-02,  6.52039051e-02,  2.14238092e-02, -9.28832311e-03,
       -3.81165706e-02, -3.52043137e-02, -2.41341861e-03, -3.61703932e-02,
        3.01193818e-02, -4.70952084e-03,  2.05624364e-02,  3.62936198e-03,
       -5.04270047e-02, -

In [15]:

import torch
from torch import nn

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=5, bias=True)
  )
)


In [16]:
def train_fn(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = torch.from_numpy(X)
        y = torch.from_numpy(np.array(y))

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 2000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_fn(list(zip(train_embedding, train_y)), model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 1.675962  [  768/   98]
Epoch 2
-------------------------------
loss: 1.657740  [  768/   98]
Epoch 3
-------------------------------
loss: 1.640371  [  768/   98]
Epoch 4
-------------------------------
loss: 1.623927  [  768/   98]
Epoch 5
-------------------------------
loss: 1.608544  [  768/   98]
Epoch 6
-------------------------------
loss: 1.593914  [  768/   98]
Epoch 7
-------------------------------
loss: 1.579911  [  768/   98]
Epoch 8
-------------------------------
loss: 1.566436  [  768/   98]
Epoch 9
-------------------------------
loss: 1.553490  [  768/   98]
Epoch 10
-------------------------------
loss: 1.541022  [  768/   98]
Epoch 11
-------------------------------
loss: 1.528963  [  768/   98]
Epoch 12
-------------------------------
loss: 1.517274  [  768/   98]
Epoch 13
-------------------------------
loss: 1.506147  [  768/   98]
Epoch 14
-------------------------------
loss: 1.495409  [  768/   98]
Epoch 15
------

In [18]:
model(torch.from_numpy(test_embedding[4]))

tensor([-5.8161, -0.1234,  8.1787,  1.7128, -4.0384], grad_fn=<ViewBackward0>)

In [19]:
test

,text,score,tdw_uuid
1,A Markov Chain model can effectively estimate ...,2,039d70cf-953f-4f6d-a5cf-b360a1bd1624
2,Incremental LTV is a more accurate measure as ...,2,9c4e4f33-284f-4040-bc29-d4a857717b42
14,The paper provides valuable recommendations on...,2,0568bebc-97ea-4af8-8308-4cb32282cbc7
20,On Testing Machine Learning Programs,2,e5287254-1433-45ec-b8dc-fc641056487f
21,On Testing Machine Learning Programs,2,0a48f8f7-d26a-4bc0-8357-9018881cebc2
23,On Testing Machine Learning Programs,2,6cebb6ff-d198-448f-bf87-7ddd8cb5d3b0
29,On Testing Machine Learning Programs,2,85ada0a0-425f-4fa5-9981-19bc2e3c0d6a
32,Customized frameworks and continuous risk asse...,1,158a1be9-dbd8-4f19-a4be-e2460fc8ff77
37,Empowering Open Data Sharing for Social Good: ...,1,d6b20068-df67-414e-9df3-c0ca34956d6e
41,Customized frameworks and continuous risk asse...,1,24a27a25-2511-4140-a417-9035fddd620c


In [20]:

from state_of_the_art.config  import config


torch.save(model.state_dict(), config.TEXT_PREDICTOR_PATH_LOCALLY)


In [29]:
from state_of_the_art.relevance_model.inference import Inference


infernece = Inference()
texts = test['text'].to_list()
actuals = test['score'].to_list()

accuracy = 0
for i, text in enumerate(texts):
    predicted = infernece.predict(text)
    if predicted == actuals[i]:
        accuracy += 1
        print(f"Correct! Predicted {predicted}  Actual: {actuals[i]}. Text: {text}")
    else:
        print(f"Fail! Predicted {predicted}  Actual: {actuals[i]}. Text: {text}")
total_accuracy  = accuracy/len(texts)
print(f"Total accuracy test set: {total_accuracy}")


/Users/jean.machado/projects/state-of-the-art-via-ai/state_of_the_art/relevance_model/inference.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_di

Correct! Predicted 2  Actual: 2. Text: A Markov Chain model can effectively estimate off-service states, using minimal data to recover off-service LTV from non-subscriber transitions.
Correct! Predicted 2  Actual: 2. Text: Incremental LTV is a more accurate measure as it accounts for potential future behaviors and re-acquisition of subscribers.
Correct! Predicted 2  Actual: 2. Text: The paper provides valuable recommendations on mitigating the risks of AI-generated images. These include embedding mandatory labels in AIGIs, enhancing public awareness of AI manipulation, and proposing that social media platforms adopt features to help users easily distinguish AI-generated images. These measures can help to prevent misinformation and protect the integrity of visual media.
Correct! Predicted 2  Actual: 2. Text: On Testing Machine Learning Programs
Correct! Predicted 2  Actual: 2. Text: On Testing Machine Learning Programs
Correct! Predicted 2  Actual: 2. Text: On Testing Machine Learning P